In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
from report_utils import excel_report

import numpy as np
from log import log

In [2]:
sql = '''
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   DATE(a.finished + 3 * interval '1 hour')
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
where a.finished at time zone 'UTC' > '2020-03-30'
and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf 
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)

'''

In [3]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-04-09 16:46:49 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-04-09 16:46:49 creating ssh tunnel to 172.16.100.19 as root...
2020-04-09 16:46:59 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 43543
2020-04-09 16:46:59 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [4]:
data = connection.query(sql)

2020-04-09 16:46:59 @telemed query:
                    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case 
				            	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
				            	else 'ПРОЧИЕ' 
				           end "name",
				    	   DATE(a.finished + 3 * interval '1 hour')
				    from appointment a
				    left join patient p on a.patient_id = p.id 
				    left join person per on p.person_id = per.id 
				    left join promotion promo on a.promotion_id = promo.id
				    where a.finished at time zone 'UTC' > '2020-03-30'
				    and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
				    and a.good
				    	and not (
				    			a.patient_id in (
				    							select patient_id from patient_categories pcat
				    						

In [5]:
connection.close()

2020-04-09 16:47:00 @telemed: closing postgres connection...
2020-04-09 16:47:00 closing ssh tunnel to 172.16.100.19...


In [156]:
df = pd.DataFrame(data)

In [157]:
df.head()

,appoint_id,person_id,phone,name,date,region
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08,Московская область
1,127919,2546091,79518565014,VEBMED,2020-03-31,Воронежская область
2,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07,Свердловская область
3,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31,Краснодарский край
4,135952,3201364,79911116364,VEBMED,2020-04-07,"Series([], )"


In [158]:
def phone_base():
    base = pd.read_excel('phone_base.xlsm')
    
    column_names = ['Region', 'Code', 'useless', 'start', 'end', 'fullstart', 'fullend', 'Region-2']
    base.columns = column_names
    
    return base

In [159]:
def get_region_by_phone(phone, base=phone_base()):
    if type(phone) != str:
        phone = str(phone)
    phone = ''.join(i for i in phone if i.isdigit())    
    if phone[0] == '8' or phone[0] == '7':
        phone = phone[1:]
    if len(phone) != 10:
        return 'Неизвестный регион'
    
    #if int(phone[0:3]) in list(base.Code.unique()):
    return base.loc[(base['Code'] == int(phone[0:3])) & 
                          (base['start'] <= int(phone[3:])) &
                          (base['end'] >= int(phone[3:]))
                         ]['Region'].to_string(index=False).strip()
    #else: 
    #    return 'Неизвестный регион'
        

----------

In [160]:
def phone_base2():
    base = pd.read_csv('DEF.csv')
    
    column_base = ['code', 'start', 'end', 'capacity', 'operator', 'region']
    base.columns = column_base
    
    return base

In [258]:
def get_region_by_phone2(phone, base=phone_base2()):
    if type(phone) != str:
        phone = str(phone)
    phone = ''.join(i for i in phone if i.isdigit())    
    if phone[0] == '8' or phone[0] == '7':
        phone = phone[1:]
    if len(phone) != 10:
        return 'Неизвестный регион'
    return base.loc[(base['code'] == int(phone[0:3])) & 
                          (base['start'] <= int(phone[3:])) &
                          (base['end'] >= int(phone[3:]))
                         ]['region'].to_string(index=False).strip()

In [10]:
#get_region_by_phone('9011009999')

In [11]:
#get_region_by_phone('+79260001133')

In [12]:
#get_region_by_phone('7(928)997-00-00')

In [13]:
#get_region_by_phone(89011009999)

In [206]:
df_test = pd.DataFrame(df.head())

In [207]:
df_test['region'] = df_test['phone'].apply(get_region_by_phone2)

In [208]:
df_test

,appoint_id,person_id,phone,name,date,region
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08,г. Москва и Московская область
1,127919,2546091,79518565014,VEBMED,2020-03-31,Воронежская обл.
2,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07,Свердловская обл.
3,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31,Краснодарский край
4,135952,3201364,79911116364,VEBMED,2020-04-07,Ставропольский край


In [209]:
df_test.loc[4]['region']

'Ставропольский край'

In [210]:
df_test.replace('Series([], )', 'Неизвестный регион')

,appoint_id,person_id,phone,name,date,region
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08,г. Москва и Московская область
1,127919,2546091,79518565014,VEBMED,2020-03-31,Воронежская обл.
2,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07,Свердловская обл.
3,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31,Краснодарский край
4,135952,3201364,79911116364,VEBMED,2020-04-07,Ставропольский край


In [259]:
df['region'] = df['phone'].apply(get_region_by_phone2)

In [260]:
df = df.replace('Series([], )', 'Неизвестный регион')

In [261]:
df.head(15)

,appoint_id,person_id,phone,name,date,region
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08,г. Москва и Московская область
1,127919,2546091,79518565014,VEBMED,2020-03-31,Воронежская обл.
2,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07,Свердловская обл.
3,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31,Краснодарский край
4,135952,3201364,79911116364,VEBMED,2020-04-07,Ставропольский край
5,127960,2626621,79149123969,ПРОЧИЕ,2020-03-31,Иркутская обл.
6,127116,2632032,79002066203,ПРОЧИЕ,2020-03-31,Свердловская обл.
7,135890,792842,79172240600,ПРОЧИЕ,2020-04-07,Республика Татарстан
8,131310,3084235,79166063464,VEBMED,2020-04-04,г. Москва и Московская область
9,127866,3056218,79608725544,ПРОЧИЕ,2020-03-31,Волгоградская обл.


In [262]:
reg_names = [name for name in df.region.unique()]

In [263]:
reg_names.sort()

In [264]:
reg_names.insert(0, 'Всего консультаций')

In [265]:
#reg_names

In [266]:
#reg_names.index('Incorrect phone')

In [267]:
d = dict()

In [268]:
for i in range(df.shape[0]):
    d[df['date'][i]] = d.get(df['date'][i], [0 for name in reg_names]) 
    d[df['date'][i]][reg_names.index(df['region'][i])] += 1
    d[df['date'][i]][0] += 1

In [269]:
#d

In [270]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()

In [271]:
report.columns = reg_names

In [272]:
report

,Всего консультаций,Алтайский край,Амурская обл.,Архангельская обл.,Архангельская область и Ненецкий автономный округ,Астраханская обл.,Белгородская обл.,Брянская обл.,Владимирская обл.,Волгоградская обл.,...,Ханты - Мансийский - Югра АО,Челябинская обл.,Чеченская Республика,Чувашская Республика,Чувашская Республика - Чувашия,Ямало-Ненецкий АО,Ярославская обл.,г. Москва и Московская область,г. Санкт-Петербург и Ленинградская область,г. Сочи|Краснодарский край
2020-03-30,526,4,2,0,0,1,8,0,7,3,...,4,8,0,1,0,3,2,172,52,0
2020-03-31,571,2,0,2,0,2,3,0,5,8,...,4,10,1,5,0,4,3,201,60,0
2020-04-01,702,3,1,4,0,0,9,2,9,9,...,8,10,2,2,0,3,11,241,81,0
2020-04-02,736,3,1,4,1,5,4,3,9,11,...,9,3,1,2,1,1,19,250,83,0
2020-04-03,843,11,2,7,3,2,6,5,6,5,...,6,11,1,2,0,2,7,285,76,1
2020-04-04,744,4,1,3,0,1,6,4,8,5,...,5,10,0,8,0,3,8,227,64,1
2020-04-05,737,5,0,2,2,1,10,5,7,11,...,6,13,2,6,0,2,14,212,69,0
2020-04-06,1001,12,3,3,1,2,8,8,16,7,...,7,17,1,10,1,6,20,288,79,0
2020-04-07,951,6,2,2,3,0,12,4,8,6,...,12,14,1,8,1,4,26,263,65,0
2020-04-08,1136,8,2,1,1,2,7,6,12,8,...,16,7,1,3,1,2,23,308,75,0


In [273]:
d2 = dict()

In [274]:
dates = [date for date in df.date.unique()]

In [275]:
dates.sort()

In [276]:
dates

[datetime.date(2020, 3, 30),
 datetime.date(2020, 3, 31),
 datetime.date(2020, 4, 1),
 datetime.date(2020, 4, 2),
 datetime.date(2020, 4, 3),
 datetime.date(2020, 4, 4),
 datetime.date(2020, 4, 5),
 datetime.date(2020, 4, 6),
 datetime.date(2020, 4, 7),
 datetime.date(2020, 4, 8)]

In [277]:
for i in range(df.shape[0]):
    d2[df['region'][i]] = d2.get(df['region'][i], [0 for date in dates]) 
    d2[df['region'][i]][dates.index(df['date'][i])] += 1
    #d2[df['region'][i]][0] += 1

In [278]:
#d2

In [279]:
report2 = pd.DataFrame.from_dict(d2, orient='index').sort_index()

In [280]:
report2.columns = dates

In [281]:
report2['total'] = report2.sum(axis=1)

In [282]:
report2.sort_values('total', ascending=False).head(25)

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,total
г. Москва и Московская область,172,201,241,250,285,227,212,288,263,308,2447
г. Санкт-Петербург и Ленинградская область,52,60,81,83,76,64,69,79,65,75,704
Республика Татарстан,10,19,14,13,24,19,64,58,42,49,312
Ивановская обл.,6,6,7,6,7,7,6,11,55,139,250
Республика Башкортостан,15,9,21,13,12,34,29,32,38,29,232
Свердловская обл.,12,16,21,28,25,27,19,25,23,24,220
Краснодарский край,16,23,23,18,23,17,18,35,16,27,216
Новосибирская обл.,18,11,10,11,11,17,12,24,29,18,161
Нижегородская обл.,16,16,15,18,14,18,12,22,15,13,159
Самарская обл.,12,10,18,16,20,18,19,14,11,14,152


In [283]:
report2 = report2.sort_values('total', ascending=False)

In [284]:
report2.to_excel('app_regions_new.xls')

----

# WEBMED

In [285]:
d3 = dict()

In [286]:
dates = [date for date in df.date.unique()]
dates.sort()

In [287]:
#df['region'][1]

In [288]:
for i in range(df.shape[0]):
    if df['name'][i] == 'VEBMED':
        #region_name = df[]
        d3[df['region'][i] + ' VEBMED'] = d3.get(df['region'][i]  + ' VEBMED', [0 for date in dates]) 
        d3[df['region'][i] + ' VEBMED'][dates.index(df['date'][i])] += 1
    #d2[df['region'][i]][0] += 1
    
    if df['name'][i] != 'VEBMED':
        d3[df['region'][i] + ' _прочие'] = d3.get(df['region'][i] + ' _прочие', [0 for date in dates]) 
        d3[df['region'][i] + ' _прочие'][dates.index(df['date'][i])] += 1

In [289]:
report3 = pd.DataFrame.from_dict(d3, orient='index').sort_index()

In [290]:
report3.columns = dates

In [291]:
report3['total'] = report3.sum(axis=1)

In [292]:
report3

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,total
Алтайский край VEBMED,0,1,2,1,6,1,2,5,3,2,23
Алтайский край _прочие,4,1,1,2,5,3,3,7,3,6,35
Амурская обл. VEBMED,0,0,1,1,2,1,0,0,0,2,7
Амурская обл. _прочие,2,0,0,0,0,0,0,3,2,0,7
Архангельская обл. VEBMED,0,0,1,0,0,1,0,2,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...
г. Москва и Московская область _прочие,157,150,173,178,207,145,112,155,152,184,1613
г. Санкт-Петербург и Ленинградская область VEBMED,1,10,18,15,16,26,22,30,27,18,183
г. Санкт-Петербург и Ленинградская область _прочие,51,50,63,68,60,38,47,49,38,57,521
г. Сочи|Краснодарский край VEBMED,0,0,0,0,1,0,0,0,0,0,1


In [293]:
report3.to_excel('app_regions_new-3.xls')

In [295]:
file_name = 'appointments_regions.xls'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
report3.to_excel(writer, sheet_name='WEB partition')
report2.to_excel(writer, sheet_name='Regions total')

writer.save()